# NSE Options Trading Analysis
## Strategy 1: Short Straddle

Identify the mathematically best strike prices for selling both CALL and PUT options.

In [12]:
# Import required libraries
from nse_api import NSEDataFetcher
import pandas as pd
from time import sleep
from IPython.display import clear_output
from datetime import datetime
from collections import defaultdict
import matplotlib.pyplot as plt

# Configuration
stock_list = ['PNB', 'BHEL', 'NTPC', 'BEL', 'IOC', 'TATASTEEL']
chosenmonths = ['Dec']
sort_by = 'ROI'  # 'Normal' or 'ROI'
atm_range_lower = 0.98  # Lower bound for ATM range (98% of current price)
atm_range_upper = 1.05  # Upper bound for ATM range (105% of current price)
margin = 0.25  # Base margin requirement (Short Straddle uses 2x this, Covered Call uses this directly)

print("✓ Libraries imported")
print(f"✓ Analyzing: {', '.join(stock_list)}")
print(f"✓ Expiry months: {', '.join(chosenmonths)}")
print(f"✓ ATM Range: {atm_range_lower*100:.0f}% - {atm_range_upper*100:.0f}% of current price")
print(f"✓ Margin: {margin*100:.0f}% (Short Straddle: {margin*2*100:.0f}%)")

✓ Libraries imported
✓ Analyzing: PNB, BHEL, NTPC, BEL, IOC, TATASTEEL
✓ Expiry months: Dec
✓ ATM Range: 98% - 105% of current price
✓ Margin: 25% (Short Straddle: 50%)


In [13]:
# Initialize NSE Data Fetcher
fetcher = NSEDataFetcher()
print("✓ NSE API connected")

Using today's cached lot sizes
✓ Loaded 213 lot sizes from cache
✓ NSE API connected


In [14]:
# Strategy 1: Short Straddle Analysis
# This runs continuously and refreshes every 15 seconds

while True:
    all_opportunities = []
    
    # Fetch data for each stock
    for symbol in stock_list:
        try:
            # Get current price and options
            current_price = fetcher.get_stock_price(symbol)
            lot_size = fetcher.get_lot_size(symbol)
            
            if current_price == 0:
                continue
            
            # Get options for chosen months
            for month in chosenmonths:
                options = fetcher.get_options_data(symbol, expiry_month=month)
                
                # Group options by strike price
                strikes = defaultdict(lambda: {'CE': None, 'PE': None})
                
                for opt in options:
                    strike = opt['strike']
                    opt_type = opt['option_type']
                    
                    # Only consider strikes within ATM range
                    if (atm_range_lower * current_price) <= strike <= (atm_range_upper * current_price):
                        strikes[strike][opt_type] = opt
                
                # Find strikes with both CALL (CE) and PUT (PE)
                for strike, options_pair in strikes.items():
                    ce_opt = options_pair['CE']
                    pe_opt = options_pair['PE']
                    
                    # Skip if either option is missing
                    if not ce_opt or not pe_opt:
                        continue
                    
                    # Extract data
                    call_premium = ce_opt['last_price']
                    put_premium = pe_opt['last_price']
                    call_volume = ce_opt['volume']
                    put_volume = pe_opt['volume']
                    expiry_full = ce_opt['expiry_date']
                    
                    # Format expiry date (remove year): "30-Dec-2025" -> "30-Dec"
                    expiry = '-'.join(expiry_full.split('-')[:2]) if expiry_full else ''
                    
                    # Skip if premiums are zero
                    if call_premium == 0 or put_premium == 0:
                        continue
                    
                    # Calculate metrics
                    combined_premium = call_premium + put_premium
                    investment = margin * 2 * lot_size * current_price  # 2x margin for short straddle
                    max_profit = combined_premium * lot_size
                    max_roi = (max_profit / investment) * 100
                    
                    # Safety ranges (breakeven points)
                    short_safety = strike - combined_premium  # Lower breakeven
                    long_safety = strike + combined_premium   # Upper breakeven
                    
                    all_opportunities.append({
                        'Symbol': symbol,
                        'Current': current_price,
                        'Strike': strike,
                        'Expiry': expiry,
                        'CALL': call_premium,
                        'PUT': put_premium,
                        'C+P': round(combined_premium, 2),
                        'Investment': int(investment),
                        'MAX Profit': int(max_profit),
                        'MAX ROI %': round(max_roi, 2),
                        'Short Safety': round(short_safety, 2),
                        'Long Safety': round(long_safety, 2),
                        'CALL Vol': call_volume,
                        'PUT Vol': put_volume
                    })
        
        except Exception as e:
            print(f"⚠️  Error processing {symbol}: {e}")
    
    # Clear output and display results
    clear_output(wait=True)
    
    print("=" * 100)
    print(" " * 35 + "SHORT STRADDLE ANALYSIS")
    print("=" * 100)
    print(f"Last Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("-" * 100)
    
    # Create DataFrame
    if all_opportunities:
        df = pd.DataFrame(all_opportunities)
        
        # Sort by ROI (highest first) or normal
        if sort_by == 'ROI':
            df = df.sort_values('MAX ROI %', ascending=False)
        else:
            df = df.sort_values(['Symbol', 'Strike'])
        
        df = df.reset_index(drop=True)
                
        display_cols = ['Symbol', 'Expiry', 'Current', 'Strike', 'Short Safety', 'Long Safety', 'CALL', 'PUT', 'C+P', 
                       'Investment', 'MAX Profit', 'MAX ROI %', 'CALL Vol', 'PUT Vol']
        print(df[display_cols].to_string(index=True))
    
    else:
        print("\n⚠️  No opportunities found. Market may be closed.")
    
    print("-" * 100)

    sleep(15)

                                   SHORT STRADDLE ANALYSIS
Last Updated: 2025-12-29 09:09:18
----------------------------------------------------------------------------------------------------
   Symbol  Expiry  Current  Strike  Short Safety  Long Safety  CALL    PUT    C+P  Investment  MAX Profit  MAX ROI %  CALL Vol  PUT Vol
0     PNB  30-Dec   120.60   126.0        119.98       132.02  0.02   6.00   6.02      482400       48160       9.98       327       13
1    BHEL  30-Dec   279.00   292.5        278.95       306.05  0.05  13.50  13.55      366187       35568       9.71       540        5
2    NTPC  30-Dec   325.65   340.0        324.70       355.30  0.10  15.20  15.30      244237       22950       9.40       286       59
3     BEL  30-Dec   393.55   410.0        393.20       426.80  0.10  16.70  16.80      280404       23940       8.54     18298      831
4    BHEL  30-Dec   279.00   290.0        278.70       301.30  0.05  11.25  11.30      366187       29662       8.10      3883

KeyboardInterrupt: 

## Strategy 2: Covered Call

Sell a CALL option while holding the underlying stock (i.e. buy the underlying stock).
Ideal for Moderate Bullish Outlook.

In [15]:
# Strategy 2: Covered Call Analysis
# This runs continuously and refreshes every 15 seconds

while True:
    all_opportunities2 = []
    
    # Fetch data for each stock
    for symbol in stock_list:
        try:
            # Get current price and options
            current_price = fetcher.get_stock_price(symbol)
            lot_size = fetcher.get_lot_size(symbol)
            
            if current_price == 0:
                continue
            
            # Get options for chosen months
            for month in chosenmonths:
                options = fetcher.get_options_data(symbol, expiry_month=month)
                
                # Only look at CALL options
                for opt in options:
                    if opt['option_type'] != 'CE':
                        continue
                    
                    strike = opt['strike']
                    
                    # Only consider strikes at or above current price (OTM/ATM calls)
                    if not (0.999 * current_price <= strike <= atm_range_upper * current_price):
                        continue
                    
                    # Extract data
                    call_premium = opt['last_price']
                    call_volume = opt['volume']
                    expiry_full = opt['expiry_date']
                    
                    # Format expiry date
                    expiry = '-'.join(expiry_full.split('-')[:2]) if expiry_full else ''
                    
                    # Skip if premium is zero
                    if call_premium == 0:
                        continue
                    
                    # Calculate metrics for covered call
                    # Investment = stock purchase cost (with margin)
                    investment = int(margin * lot_size * current_price)
                    interest = round(0.01 * margin * lot_size * current_price, 2)  # Holding cost
                    
                    # Max Profit: if stock rises to strike + premium collected - interest
                    max_profit = int(((strike - current_price + call_premium) * lot_size) - interest)
                    max_roi = round(100 * (((strike - current_price + call_premium) * lot_size) - interest) / investment, 2)
                    
                    # Safety Point: price at which you break even
                    safety_point = round((1.003 * current_price) - call_premium, 2)
                    safety_pct = round(((call_premium - (0.003 * current_price)) / current_price * 100), 2)
                    
                    all_opportunities2.append({
                        'Symbol': symbol,
                        'Current': current_price,
                        'Strike': strike,
                        'Expiry': expiry,
                        'Safety Point': safety_point,
                        'Safety %': safety_pct,
                        'CALL': call_premium,
                        'Investment': investment,
                        'MAX Profit': max_profit,
                        'MAX ROI %': max_roi,
                        'CALL Vol': call_volume
                    })
        
        except Exception as e:
            print(f"⚠️  Error processing {symbol}: {e}")
    
    # Clear output and display results
    clear_output(wait=True)
    
    print("=" * 100)
    print(" " * 35 + "COVERED CALL ANALYSIS")
    print("=" * 100)
    print(f"Last Updated: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
    print("-" * 100)
    
    # Create DataFrame
    if all_opportunities2:
        df2 = pd.DataFrame(all_opportunities2)
        
        # Sort by ROI
        df2 = df2.sort_values('MAX ROI %', ascending=False)
        df2 = df2.reset_index(drop=True)
                
        display_cols = ['Symbol', 'Expiry', 'Current', 'Strike', 'Safety Point', 'Safety %', 'CALL', 
                       'Investment', 'MAX Profit', 'MAX ROI %', 'CALL Vol']
        print(df2[display_cols].to_string(index=True))
    
    else:
        print("\n⚠️  No covered call opportunities found.")
    
    print("-" * 100)

    sleep(15)

                                   COVERED CALL ANALYSIS
Last Updated: 2025-12-29 09:09:50
----------------------------------------------------------------------------------------------------
       Symbol  Expiry  Current  Strike  Safety Point  Safety %  CALL  Investment  MAX Profit  MAX ROI %  CALL Vol
0         IOC  30-Dec   162.10   170.0        162.57     -0.29  0.02      197559       36634      18.54       327
1        BHEL  30-Dec   279.00   292.5        279.79     -0.28  0.05      183093       33737      18.43       540
2   TATASTEEL  30-Dec   172.30   180.0        172.77     -0.27  0.05      236912       40255      16.99     11172
3         PNB  30-Dec   120.60   126.0        120.94     -0.28  0.02      241200       40948      16.98       327
4        NTPC  30-Dec   325.65   340.0        326.53     -0.27  0.10      122118       20453      16.75       286
5         IOC  30-Dec   162.10   169.0        162.56     -0.28  0.03      197559       31808      16.10       239
6         

KeyboardInterrupt: 